## GICS 체계로 클러스터링

In [ ]:
import pandas as pd

# 엑셀 파일 경로
excel_file_path = '/Users/sungsupark/Desktop/2025-1 수업자료/소프트웨어융합캡스톤디자인/종목_GICS분류_클러스터.xlsx'  # 실제 경로로 변경하세요

# 엑셀 파일 읽기
df = pd.read_excel(excel_file_path)

# CSV 파일 경로
csv_file_path = '/Users/sungsupark/Desktop/2025-1 수업자료/소프트웨어융합캡스톤디자인/csv/종목_GICS분류_클러스터.csv'  # 저장할 CSV 파일 경로

# CSV 파일로 저장
df.to_csv(csv_file_path, index=False, encoding='utf-8')

print(f"엑셀 파일이 성공적으로 CSV 파일로 변환되었습니다: {csv_file_path}")

엑셀 파일이 성공적으로 CSV 파일로 변환되었습니다: /Users/sungsupark/Desktop/2025-1 수업자료/소프트웨어융합캡스톤디자인/csv/종목_GICS분류_클러스터.csv


In [ ]:
df_processing_cospi200 = pd.read_csv('/Users/sungsupark/Desktop/2025-1 수업자료/소프트웨어융합캡스톤디자인/csv/financial_data_processing_cospi200.csv')
df_gics = pd.read_csv('/Users/sungsupark/Desktop/2025-1 수업자료/소프트웨어융합캡스톤디자인/csv/종목_GICS분류_클러스터.csv')


In [9]:
df_gics
df_processing_cospi200

,기업명_그룹,매출액,영업이익,당기순이익,자산총계,자본총계,자본금,영업활동현금흐름,이자발생부채,영업이익률,순이익률,ROE(%),ROA(%),부채비율,EPS(원),PER(배),BPS(원),PBR(배),발행주식수(보통주)
0,BGF리테일,78168.10,2353.90,1806.20,30413.75,10074.50,173.00,6499.40,3371.05,3.0045,2.3025,19.1640,6.1845,204.4665,10452.15,13.842000,58318.90,2.5360,17283906.0
1,BNK금융지주,82983.75,9329.10,7427.55,1402323.10,106667.95,16297.00,1059.20,194600.65,12.1160,9.5230,7.2460,0.5505,1212.4140,2177.75,3.688000,31087.35,0.2545,323323988.3
2,CJ,399896.75,21261.65,4553.45,459940.30,173529.85,1790.00,43187.50,182317.25,5.2905,1.1735,3.2850,1.0215,164.8735,4743.30,23.256000,157376.10,0.5830,29176998.0
3,CJ대한통운,117828.20,4457.35,2185.80,95146.65,40917.65,1141.00,6244.35,33233.10,3.7705,1.8440,5.1870,2.3095,132.5530,8222.50,18.852000,183922.25,0.6120,22812344.0
4,CJ제일제당,284474.55,14863.60,6214.55,290387.75,115612.75,819.00,20663.90,110318.45,5.2365,2.2195,6.7945,2.2100,151.2490,25676.20,16.468500,433280.45,0.7875,15054186.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,현대해상,168437.95,10420.80,7558.30,476717.70,60701.60,447.00,14037.90,16655.60,6.3260,4.5935,12.1730,1.5880,746.7925,8454.75,3.715000,77333.60,0.3630,89400000.0
191,호텔신라,39464.80,361.90,-312.55,31713.35,7943.10,2000.00,1265.10,16232.85,0.8090,-0.8505,-3.0005,-0.9120,336.9375,-781.90,3.286643,20980.10,3.7785,39248121.0
192,효성중공업,40082.75,2242.65,1152.10,49684.20,13957.55,466.00,2791.35,12929.00,5.2690,2.6040,8.2110,2.2965,269.5100,11180.00,21.237749,135929.25,1.2290,9324548.0
193,효성티앤씨,77966.55,3994.95,2516.70,42602.50,15773.25,216.00,5586.25,13711.75,5.0295,3.1635,18.9360,6.5965,177.7275,45108.25,33.786000,295216.55,1.1600,4327682.0


### Davies-Bouldin Index 계산

In [12]:
from sklearn.metrics import davies_bouldin_score

# 컬럼명 맞춰 병합 준비
df_gics_renamed = df_gics.rename(columns={'종목명': '기업명_그룹'})

# '기업명_그룹'을 기준으로 병합
df_merged = pd.merge(df_processing_cospi200, df_gics_renamed[['기업명_그룹', 'Cluster']], on='기업명_그룹')

# 클러스터 레이블과 특징 데이터 분리
labels = df_merged['Cluster']
features = df_merged.drop(columns=['기업명_그룹', 'Cluster'])

# Davies-Bouldin Index 계산
db_index = davies_bouldin_score(features, labels)

# 결과 출력
print("Davies-Bouldin Index:", db_index)

Davies-Bouldin Index: 19.687756301247756


### Davies-Bouldin Index이 19.69로 나와 기존에 분류 체계로 분류하면 클러스터링 결과가 좋지 않은 것을 확인할 수 있다